In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## ----------------------- please set up data path echoDynamic.zip --------------------------------------------
## --------------------------- paths where models are going to be saved ---------------------------------------


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os,zipfile,keras
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Major_Project/EchoNet-Dynamic.zip', 'r')
zip_ref.extractall('/content/tmp')
zip_ref.close()

In [ ]:
import pandas as pd
import numpy as np 
import cv2
import keras
from keras.layers import Conv2D,Input,LeakyReLU,BatchNormalization,Dropout,Concatenate,Dropout,GlobalAveragePooling2D,Dense,Flatten
from keras.models import Model
from keras.utils.vis_utils import plot_model
from keras.optimizers import Adam
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import random 
PATH = {
    'data_path' : "/content/tmp/EchoNet-Dynamic/", 
    'save_model': '/content/drive/MyDrive/Major Project/',
    'figure_path' : '/content/drive/MyDrive/Major_Project/Figure/'
}


# ------------------------------------------------------- global DATA --------------------------
# ------------------------------------ this will cause video data to erase --------------------
# -------------------------------------- so think before execution ---------------------------

TRAIN_SAMPLE_LOW = 0
TRAIN_SAMPLE_HIGH = 7000 ## make it 6K 
TEST_SAMPLE_LOW = 7000 ## 6K
TEST_SAMPLE_HIGH = 10000 ## 10K 
COMPRESSOR_EPOCHS = 10
PREDICTOR_EPOCHS = 10

fileList = pd.read_csv(PATH['data_path'] + 'FileList.csv')
inputSize = TRAIN_SAMPLE_HIGH - TRAIN_SAMPLE_LOW + 1
videoArrays = []
EF = []

In [ ]:
## -------------------------------------------------------------- preprocessing -----------------------------------------------------------

def VideoPreprocess(PATH,lower,upper):
  ## EF and videoArrays generated
  for i in range(lower,upper): 
    dfRow = fileList.iloc[i]
    fileName = dfRow.FileName
    path = PATH['data_path'] + "/Videos/" + fileName + '.avi'
    cap = cv2.VideoCapture(path)
    frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if(frameCount < 100):
       continue
    buf = np.empty((112, 112,frameCount), np.dtype('uint8')) ### creating 50 frames size buffer
    frameId = 0
    for frameId in range(frameCount): 
      ret,frame = cap.read()
      grayScale = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
      buf[:,:,frameId] = grayScale
      del frame 
    videoArrays.append(buf)
    EF.append(dfRow.EF)
  return 

# --------------------------------------------- Layers ------------------------------------------


def convS(input,filters,batch,stride,drop = 0.0): 
  ip = Input(shape = input.shape[1:])
  output = Conv2D(filters = filters,kernel_size = (3,3),strides=stride)(ip)
  if batch: 
    output = BatchNormalization()(output)
  output = LeakyReLU(alpha=.2)(output)
  if drop: 
    output = Dropout(drop)(output)
  model = Model(ip,output)
  return model(input)
def concatenatedLayers(input1,input2,filters,batch,drop):
  ip1 = Input(shape = input1.shape[1:])
  ip2 = Input(shape = input2.shape[1:])
  concat = Concatenate()([ip1,ip1])
  output = Conv2D(filters = filters,kernel_size = (3,3),strides=(1,1),padding='same')(concat)
  if batch: 
    output = BatchNormalization()(output)
  output = LeakyReLU(alpha=.2)(output)
  if drop:
    output = Dropout(drop)(output)
  model = Model([ip1,ip2],output)
  return model([input1,input2])
def convLayer(input,filters,batch,drop): 
  ip = Input(shape = input.shape[1:])
  output = Conv2D(filters = filters,kernel_size = (3,3),strides=(1,1),padding='same')(ip)
  if batch: 
    output = BatchNormalization()(output)
  output = LeakyReLU(alpha=.2)(output)
  if drop:
    output = Dropout(drop)(output)
  model = Model(ip,output)
  return model(input)

## -------------------------------------------------------------------- predictor generator ----------------------------------------------------


class predictorDataGenerator(keras.utils.Sequence):
  'Generates data for Keras'
  def __init__(self, PATH):
    'Initialization'
    self.batch_size = 1
    self.len = len(videoArrays) ## global fileList 
    self.on_epoch_end()  
    self.index = 0

  def __len__(self):
    'Denotes the number of batches per epoch'
    return self.len 

  def __getitem__(self, index):
    'Generate one batch of data'
    X, y = self.__data_generation(self.index)
    return X, y

  def on_epoch_end(self):
    'Updates indexes after each epoch'
    self.index = random.randrange(0,self.len-1)

  def __data_generation(self, index):
    offset = 20
    video = videoArrays[index]
    video = (video - 127.5)/127.5 
    count = video.shape[0] - 50
    predictorData = np.zeros(((count)//offset,112,112,2))
    y = np.zeros(((count)//offset,1))
    for i in range(0,count - offset + 1,offset): 
      id = i//offset
      x = video[:,:,i:i+50] 
      if(x.shape[2] != 50): 
        del x 
        predictorData[id] = predictorData[id-1]
        y[id] = y[id-1] 
        continue
      image = x[:,:,1:2]
      image = np.expand_dims(image,axis = 0)
      x = np.expand_dims(x,axis = 0) 
      generatedImage = gen.predict(x,verbose = 0)
      predictorData[id] = np.concatenate([generatedImage,image],axis = 3)[0] # single BATcH
      y[id] = np.array([EF[index]])
      del x,image,generatedImage  
    del video
    return predictorData,y/100

# ------------------------------------------- predictor MODEL -----------------------------------------------

def predictor(lr = 0.001): 
  input = Input((112,112,2)) 
  output = convS(input,filters = 16,batch = False,stride = (1,1))
  output = convS(output,filters = 32,batch = False,stride = (1,1))
  output = convS(output,filters = 64,batch = True,stride = (2,2),drop = .4)
  output = convS(output,filters = 128,batch = True,stride = (2,2),drop = .4)
  output = convS(output,filters = 32,batch = True,stride = (1,1))
  output = Dropout(.2)(output)
  output = convS(output,filters = 16,batch = False,stride = (2,2))
  output = Dropout(.1)(output)
  output = convS(output,filters = 8,batch = False,stride = (2,2))  
  output = Dropout(.4)(Flatten()(output))
  output = Dense(64,activation = "relu")(output)
  output = Dense(1,activation = 'sigmoid')(output)
  model = Model(input,output)
  model.compile(optimizer = Adam(learning_rate = lr),loss = 'mae')
  return model

# -------------------------------------------- testing predictor MODEL 


def finalPredictions(PATH,low,high): 
  real_ = []
  pred_ = []
  for index in range(low,high): 
    video = videoArrays[index]
    video = (video - 127.5)/127.5 
    predictorData = np.empty((video.shape[0]//50,112,112,2))
    y = np.empty((video.shape[0]//50,1))
    i = 0
    while(i + 50 < video.shape[0]): 
      id = i//50
      x = video[:,:,i:i+50]
      image = x[:,:,1:2]
      image = np.expand_dims(image,axis = 0)
      x = np.expand_dims(x,axis = 0) 
      generatedImage = gen.predict(x,verbose = 0)
      predictorData[id] = np.concatenate([generatedImage,image],axis = 3)[0]
      y[id] = np.array([EF[index]])
      i += 50
      del generatedImage,x,image
    predZ = pred.predict(predictorData,verbose = 0)
    predZ = predZ.reshape((-1,))
    y = y.reshape((-1,))/100
    real_.append(y.tolist())
    pred_.append(predZ.tolist())
    del predictorData,video
  return real_,pred_

def evaluate(real_input,predicted_output): 
  r2 = r2_score(real_input,predicted_output)
  mae = mean_absolute_error(real_input,predicted_output)
  mse = mean_squared_error(real_input, predicted_output)
  return mae,mse,r2

def distr(arr): 
  min = np.amin(arr)
  max = np.amax(arr)
  range = np.ptp(arr)
  variance = np.var(arr)
  sd = np.std(arr)
  print("Measures of Dispersion")
  print("Minimum =", min)
  print("Maximum =", max)
  print("Range =", range)
  print("Variance =", variance)
  print("Standard Deviation =", sd)
gen = keras.models.load_model(PATH['save_model'] + 'wholeTestingGen.h5')

In [ ]:
## -------------------------------------------- video.npy generation ------------------------------------------
VideoPreprocess(PATH,0,3000) 

In [ ]:
#---------------------------------------- trainign predictor----------------------------------
predGen = predictorDataGenerator(PATH)
pred = predictor(lr = .0001)
pred.fit_generator(predGen,epochs = 2,workers = 4)

<ipython-input-7-3e1e3b2be43d>:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  pred.fit_generator(predGen,epochs = PREDICTOR_EPOCHS,workers = 4)


Epoch 1/10
4853/4853 [==============================] - 726s 149ms/step - loss: 0.0313
Epoch 2/10
4853/4853 [==============================] - 716s 147ms/step - loss: 0.0065
Epoch 3/10
  89/4853 [..............................] - ETA: 11:57 - loss: 0.0060

KeyboardInterrupt: ignored

In [ ]:
pred = keras.models.load_model(PATH['save_model'] + 'wholeTestingPred.h5')

In [ ]:
# --------------------------------------- checking training accuracy ---------------------------------
predicted,real = finalPredictions(PATH,0,1000)
mae,mse,r2 = evaluate(real,predicted)
print(mae,mse,r2)
real = np.array(real)
predicted = np.array(predicted)
np.save(PATH['save_model'] + 'wholeTesting3000.npy',real)
np.save(PATH['save_model'] + 'wholeTesting3000.npy',predicted)

0.08384157093355166 0.014679279731813873 -27951.6979648494


In [ ]:
# --------------------------------- Creating Testing DAta -----------------------
del videoArrays, EF
videoArrays = []
EF = []
VideoPreprocess(PATH,2000,5000) 

In [ ]:
# -------------------------------------- testing ----------------------------------------------------
predicted,real = finalPredictions(PATH,1000,2000)
mae,mse,r2 = evaluate(real,predicted)
print(mae,mse,r2)
print(' real ')
distr(real)
print(" predicted ")
distr(predicted)
real = np.array(real)
predicted = np.array(predicted)
np.save(PATH['save_model'] + 'wholeTestingReal3000Test.npy',real)
np.save(PATH['save_model'] + 'wholeTestingPredicted3000Test.npy',predicted)

0.08583315567228658 0.015923415619405773 -33785.2844831806
 real 
Measures of Dispersion
Minimum = 0.5796144604682922
Maximum = 0.584489643573761
Range = 0.00487518310546875
Variance = 4.717075034382736e-07
Standard Deviation = 0.0006868096559005804
 predicted 
Measures of Dispersion
Minimum = 0.1021196939
Maximum = 0.8600968401
Range = 0.7579771462
Variance = 0.015290731738566953
Standard Deviation = 0.1236556983667431


In [ ]:
#------------------------------- training on testing data itself -------------------
predGen = predictorDataGenerator(PATH)
pred.fit_generator(predGen,epochs = 2,workers = 4)
pred.save(PATH['save_model'] + 'wholeTestingPred5000.h5')

<ipython-input-11-81caec03f2ce>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  pred.fit_generator(predGen,epochs = 2,workers = 4)


Epoch 1/2
2915/2915 [==============================] - 446s 152ms/step - loss: 0.0071
Epoch 2/2
2915/2915 [==============================] - 451s 154ms/step - loss: 0.0016


In [ ]:
#-------------------------------- test again on testing data -------------------------
del videoArrays, EF
videoArrays = []
EF = []
VideoPreprocess(PATH,4000,7000) 

In [ ]:
# ---------------- whole data trained model ------------------------------------
predicted,real = finalPredictions(PATH,1000,2000)
mae,mse,r2 = evaluate(real,predicted)
print(mae,mse,r2)
real = np.array(real)
predicted = np.array(predicted)
np.save(PATH['save_model'] + 'wholeTestingReal5000Test.npy',real)
np.save(PATH['save_model'] + 'wholeTestingPredicted5000Test.npy',predicted)

0.09220274870740039 0.02004206688379119 -1705181.7434638636


In [ ]:
predGen = predictorDataGenerator(PATH)
pred.fit_generator(predGen,epochs = 1,workers = 4)
pred.save(PATH['save_model'] + 'wholeTestingPred7000.h5')

<ipython-input-14-4d9243eb0223>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  pred.fit_generator(predGen,epochs = 1,workers = 4)


2894/2894 [==============================] - 457s 158ms/step - loss: 3.4320e-04


In [ ]:
del videoArrays, EF
videoArrays = []
EF = []
VideoPreprocess(PATH,7000,10000) 

In [ ]:
predicted,real = finalPredictions(PATH,0,1000)
mae,mse,r2 = evaluate(real,predicted)
print(mae,mse,r2)
real = np.array(real)
predicted = np.array(predicted)
np.save(PATH['save_model'] + 'wholeTestingReal7000.npy',real)
np.save(PATH['save_model'] + 'wholeTestingPredicted7000.npy',predicted)

0.08609048656691397 0.016633046342348287 -31238274.140028514


In [ ]:
del videoArrays, EF
videoArrays = []
EF = []
VideoPreprocess(PATH,7000,10000) 

In [ ]:
predGen = predictorDataGenerator(PATH)
pred.fit_generator(predGen,epochs = 2,workers = 4)
pred.save(PATH['save_model'] + 'wholeTestingPred7000.h5')

<ipython-input-22-34fe33c7104c>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  pred.fit_generator(predGen,epochs = 2,workers = 4)


Epoch 1/2
 480/2891 [===>..........................] - ETA: 6:08 - loss: 0.0018

KeyboardInterrupt: ignored

In [ ]:
pred = keras.models.load_model(PATH['save_model'] + 'wholeTestingPred.h5')

In [ ]:
VideoPreprocess(PATH,0,6000)

In [ ]:
predGen = predictorDataGenerator(PATH)
pred.fit_generator(predGen,epochs = 2,workers = 4)
pred.save(PATH['save_model'] + 'wholeTestingPred10000.h5')

<ipython-input-6-e896cb37c47b>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  pred.fit_generator(predGen,epochs = 2,workers = 4)


Epoch 1/2
1141/5814 [====>.........................] - ETA: 11:52 - loss: 0.0187

KeyboardInterrupt: ignored

In [ ]:
del videoArrays, EF
videoArrays = []
EF = []
VideoPreprocess(PATH,7000,8000) 

In [ ]:
predicted,real = finalPredictions(PATH,0,700)
mae,mse,r2 = evaluate(real,predicted)
print(mae,mse,r2)
print(' real ')
distr(real)
print(" predicted ")
distr(predicted)
real = np.array(real)
predicted = np.array(predicted)
np.save(PATH['save_model'] + 'wholeTestingReal1000.npy',real)
np.save(PATH['save_model'] + 'wholeTestingPredicted1000.npy',predicted)

0.21323556447229236 0.05746213116522644 -340.30641775312495
 real 
Measures of Dispersion
Minimum = 0.7325729131698608
Maximum = 0.8111540675163269
Range = 0.07858115434646606
Variance = 0.00016840135722145924
Standard Deviation = 0.012976954851638316
 predicted 
Measures of Dispersion
Minimum = 0.07719652916
Maximum = 0.7566604788
Range = 0.67946394964
Variance = 0.012079271964545966
Standard Deviation = 0.10990574127199164
